# toorPIA APIテスト用ノートブック

本ノートブックはGoogle Colabで実行できるように作成されています。

サンプルデータ（`biopsy.csv`、`biopsy-add.csv`）は自動的にGitHubから取得されます。

## toorPIA APIライブラリのインストール

Google Colabでは、以下のセルを実行してtoorPIA APIライブラリをインストールしてください。
他のpip環境でも同様にインストール可能です。

In [ ]:
!pip install git+https://github.com/toorpia/toorpia.git

## APIキーの設定方法

次にAPIキーを設定します。以下の優先順位で自動的に設定されます：

### 方法1: Google Colab Secrets機能の使用（推奨、Colab環境）
1. 左側のパネルから🔑（鍵マーク）をクリック
2. 「新しいシークレット」をクリック
3. 名前に `TOORPIA_API_KEY`、値に実際のAPIキーを入力
4. 保存

### 方法2: 既存の環境変数を使用（ローカル環境）
事前にターミナルで以下を実行しておくと、自動的に読み込まれます：
```bash
export TOORPIA_API_KEY="your_actual_api_key"
```

### 方法3: コードに直接記述
上記が利用できない場合、次のセルの `'Valid_API_KEY'` を実際のAPIキーに置き換えてください。

In [ ]:
# APIキーの設定
import os

# 方法1: Google Colab Secrets機能を使用（推奨）
try:
    from google.colab import userdata
    api_key = userdata.get('TOORPIA_API_KEY')
    os.environ['TOORPIA_API_KEY'] = api_key
    print("Google Colab SecretsからAPIキーを設定しました")
except:
    # 方法2: 既存の環境変数をチェック
    if 'TOORPIA_API_KEY' in os.environ and os.environ['TOORPIA_API_KEY']:
        print("既存の環境変数からAPIキーを読み込みました")
    else:
        # 方法3: 環境変数で直接設定
        os.environ['TOORPIA_API_KEY'] = 'Valid_API_KEY'  # ここに実際のAPIキーを入力
        print("環境変数にAPIキーを設定しました")
        print("注意: 実際のAPIキーに置き換えてください")

## toorPIAライブラリの初期化

APIキー設定後、toorPIAクライアントを初期化します。

In [ ]:
from toorpia import toorPIA

try:
    tp = toorPIA()
    print("toorPIA クライアントの初期化が完了しました")
except Exception as e:
    print(f"エラー: {e}")
    print("APIキーが正しく設定されているか確認してください")

## toorPIA APIによる解析例

ここでは、`biopsy.csv`を用いてtoorPIAのベースマップを作成し、そのあと、このベースマップに対して`biopsy-add.csv`を追加プロットする方法を例示します。

### サンプルデータの読み込み

toorPIA APIは、pandasのDataFrame形式のデータを解析できます。
サンプルデータはGitHubから自動的に取得され、解析に不要な項目（No, ID, Diagnosis）は除外されます。

In [ ]:
import pandas as pd

# GitHubから直接データを読み込み
try:
    df = pd.read_csv('https://raw.githubusercontent.com/toorpia/toorpia/master/samples/biopsy.csv')
    df = df.drop(columns=["No", "ID", "Diagnosis"])
    print("データの読み込みが完了しました")
    df
except Exception as e:
    print(f"データの読み込みに失敗しました: {e}")
    print("インターネット接続を確認してください")

### `fit_transform()`関数によるベースマップ作成

次のセルのように実行することでベースマップを作成することができます。
返り値はNumPy Array形式で、ここにベースマップの座標値が含まれます。

In [ ]:
res = tp.fit_transform(df)
res

In [ ]:
# ベースマップのshareURLを表示
if hasattr(tp, 'shareUrl') and tp.shareUrl:
    from IPython.display import display, HTML
    print("ベースマップの共有URL:")
    display(HTML(f'<a href="{tp.shareUrl}" target="_blank">{tp.shareUrl}</a>'))
else:
    print("ShareURLが取得できませんでした")

### ベースマップのプロット

`matplotlib`を用いてベースマップのプロットを作成してみます。

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(res[:,0], res[:,1])
plt.show()

### 追加プロットの作成

追加プロットデータもGitHubから自動取得し、先ほど生成したベースマップに対してプロットします。

In [ ]:
# 追加プロットデータをGitHubから直接読み込み
try:
    df_add = pd.read_csv('https://raw.githubusercontent.com/toorpia/toorpia/master/samples/biopsy-add.csv')
    df_add = df_add.drop(columns=["No", "ID", "Diagnosis"])
    print("追加プロットデータの読み込みが完了しました")
    df_add
except Exception as e:
    print(f"追加プロットデータの読み込みに失敗しました: {e}")
    print("インターネット接続を確認してください")

次のセルにあるとおり`addplot()`メソッドで追加プロットを作成することができます。
結果には座標データの他に、異常度判定情報も含まれます。

In [ ]:
res_add = tp.addplot(df_add)

# addplotの結果から座標データを取得
if isinstance(res_add, dict) and 'xyData' in res_add:
    res_add_coords = res_add['xyData']
    print(f"異常度判定: {res_add.get('abnormalityStatus', 'unknown')}")
    if res_add.get('abnormalityScore') is not None:
        print(f"異常度スコア: {res_add.get('abnormalityScore')}")
else:
    res_add_coords = res_add  # 従来形式の場合

res_add_coords

In [ ]:
# 追加プロットのshareURLを表示
if hasattr(tp, 'shareUrl') and tp.shareUrl:
    from IPython.display import display, HTML
    print("追加プロット結果の共有URL:")
    display(HTML(f'<a href="{tp.shareUrl}" target="_blank">{tp.shareUrl}</a>'))
else:
    print("ShareURLが取得できませんでした")

In [ ]:
plt.scatter(res[:,0], res[:,1], label='Base Map')
plt.scatter(res_add_coords[:,0], res_add_coords[:,1], label='Additional Plot')
plt.legend()
plt.show()

## 補足：ベースマップのID番号の取得

`tp.fit_transform()`関数でベースマップを作成した直後に、そのベースマップのID番号を取得しておくことで、あとで、特定のベースマップに対して追加プロットをすることができます。

In [ ]:
# ベースマップの作成と、ベースマップID番号の取得
res = tp.fit_transform(df)
print("toorPIA Map ID No: ", tp.mapNo)

# ベースマップID番号を指定して追加プロット
add_res = tp.addplot(df_add, tp.mapNo)

In [ ]:
plt.scatter(res[:,0], res[:,1], label='Base Map')
plt.scatter(res_add_coords[:,0], res_add_coords[:,1], label='Additional Plot')
plt.legend()
plt.show()